# BI output results and speed comparaison
## 1. Continuous variable (model 4.3)
### Output comparaison

In [40]:
import time as tm
from src.main import*
## Model m4.3
d = pd.read_csv('./data/Howell1.csv', sep=';')
d = d[d.age > 18]
#self.df["weight.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
d.weight = d.weight - d.weight.mean()
d.age = d.age - d.age.mean()
formula = dict(main1 = 'height ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha + beta * weight',
            prior1 = 'sigma ~ Uniform(0,50)',
            prior2 = 'alpha ~ Normal(178,20)',
            prior3 = 'beta ~ Normal(0,1)')    

self = model(formula, df = d, float = 32)
start = tm.time()
self.fit(observed_data = dict(height =d.height.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)   
end = tm.time()    
self.summary(round_to = 'none')     
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:40:35.752902: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:40:35.752963: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:40:35.752976: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:40:35.753297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:40:35.753311: I tensorflow/core/common_runtime/gpu/gpu

BI took: 5.1852 seconds


In [41]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """
data{
  vector[346] height;
  vector[346] weight;
}
parameters{
  real a;
  real<lower=0> b;
  real<lower=0,upper=50> sigma;
}
model{
  vector[346] mu;
  sigma ~ uniform( 0 , 50 );
  b ~ lognormal( 0 , 1 );
  a ~ normal( 178 , 20 );
  for ( i in 1:346 ) {
    mu[i] = a + b* weight[i] ;
  }
  height ~ normal( mu , sigma );
  
}
"""
data = {
  'height': d.height.values,
  'weight': d.weight.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.1s, done.Messages from stanc:
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 15, column 20: Argument
    20 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 15, column 14: Argument
    178 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 13, column 23: Argument
    50 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    in its declaration. These hard constraints are not recommended, for two
    reasons: (a) Except when there are logical or physical constraints, it is
    very unusual for you to be sure that a parameter will fall inside a
    specified range, and (b) The infinite gradient induced by a hard


Pystan took: 11.7146 seconds


## Output comparaison

In [42]:
pd.DataFrame(
    {
        "tfp": self.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df.b.mean(), df.a.mean()]
    })

,tfp,pystan
sigma[0],5.138569,5.142233
beta[0],0.905371,0.904922
alpha[0],154.649124,154.653531


## 2. Categorical variable (model 5.9)
### Output comparaisons

In [43]:
from src.main import*
self = model()
self.import_csv('./data/milk.csv', sep = ';')
self.df["K"] = self.df["kcal.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
self.index(cols = "clade")
formula = dict(main = 'K ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha[index_clade]',
            prior1 = 'alpha~ Normal(0,0.5)',
            prior2 = 'sigma ~ Exponential(1)') 

self.f = formula

start = tm.time()
self.build_model()
self.fit(observed_data = dict(K =self.df.K.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:41:05.610587: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610635: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610650: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610885: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:05.610897: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.2587 seconds


In [44]:
import stan
import nest_asyncio
nest_asyncio.apply()
stan_code = """
data{
    vector[29] K;
    array[29] int clade_id;
}
parameters{
    vector[4] a;
    real<lower=0> sigma;
}
model{
    vector[29] mu;
    sigma ~ exponential( 1 );
    a ~ normal( 0 , 0.5 );
    for ( i in 1:29 ) {
        mu[i] = a[clade_id[i]];
    }
    K ~ normal( mu , sigma );
    
}
"""
data = {
  'clade_id': self.df.index_clade.values+1,
  'K': self.df.K.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.5s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.2 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_wva56780/model_utzoleag.stan', line 17, column 4 to column 29)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 2.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would

Pystan took: 11.8828 seconds


In [45]:
pd.DataFrame(
    {
        "tfp": self.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean()]
    })

,tfp,pystan
sigma[0],0.797389,0.800678
alpha[0],-0.463416,-0.463840
alpha[1],0.348681,0.345072
alpha[2],0.633578,0.637753
alpha[3],-0.550391,-0.546003


## 3. Continuous interactions terms (model 8.3)
### Output comparaisons

In [46]:
from src.main import*
d = pd.read_csv('./data/tulips.csv', sep = ';')
d["blooms_std"] = d.blooms / d.blooms.max()
d["water_cent"] = d.water - d.water.mean()
d["shade_cent"] = d.shade - d.shade.mean()

formula = dict(
            main = 'blooms_std ~ Normal( mu , sigma ) ',
            likelihood ='mu ~ a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent' ,
            prior1 = 'a ~ Normal( 0.5 , 0.25 ) ',
            prior2 = 'bw ~ Normal( 0 , 0.25 ) ',
            prior3 = 'bs ~ Normal( 0 , 0.25 ) ',
            prior4 = 'bws ~ Normal( 0 , 0.25 ) ',
            prior5 = 'sigma ~ Exponential( 1 )',
            )
start = tm.time()
m8_5 = model(formula, d)
m8_5.fit(observed_data = dict(blooms_std =d.blooms_std.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:41:28.892398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892472: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892659: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:28.892672: I tensorflow/core/common_runtime/gpu/gpu

BI took: 7.1625 seconds


In [47]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    vector[27] blooms_std;
    array[27] int shade_cent;
    array[27] int water_cent;
}
parameters{
    real a;
    real bw;
    real bs;
    real bws;
    real<lower=0> sigma;
}
model{
    vector[27] mu;
    sigma ~ exponential( 1 );
    bws ~ normal( 0 , 0.25 );
    bs ~ normal( 0 , 0.25 );
    bw ~ normal( 0 , 0.25 );
    a ~ normal( 0.5 , 0.25 );
    for ( i in 1:27 ) {
        mu[i] = a + bw * water_cent[i] + bs * shade_cent[i] + bws * water_cent[i] * shade_cent[i];
    }
    
    blooms_std ~ normal( mu , sigma );
}
"""

data = {
    'blooms_std' : d["blooms_std"].values,
    "water_cent": d["water_cent"].values.astype(int),
    "shade_cent": d["shade_cent"].values.astype(int),
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.3s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_5kkvti7s/model_7gq22chx.stan', line 25, column 4 to column 38)
  If this warni

Pystan took: 11.7797 seconds


In [48]:
pd.DataFrame(
    {
        "tfp": m8_5.summary(round_to='none')['mean'],
        "pystan": [df.sigma.mean(),  df['bws'].mean(), df['bw'].mean(), df['bs'].mean(), df['a'].mean()]
    })

,tfp,pystan
sigma[0],0.095120,0.142249
bws[0],-0.223282,-0.142495
bw[0],0.010972,0.206738
bs[0],0.078516,-0.112914
a[0],0.393192,0.358084


## 4. Binomial model
### Output comparaisons

In [49]:
from src.main import*
d = pd.read_csv('./data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

d_aggregated = (
    d.groupby(["treatment", "actor", "side", "cond"])["pulled_left"].sum().reset_index()
)
d_aggregated.rename(columns={"pulled_left": "left_pulls"}, inplace=True)
d_aggregated["actor_id"] = d_aggregated["actor"].values - 1

formula = dict(
    main = 'pulled_left ~ Binomial( 1 , logits = p )' ,
    likelihood = 'p ~ a' ,
    prior1 = 'a ~ Normal( 0 , 10 )'
)
start = tm.time()
m11_1 = model(formula, d)
m11_1.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:41:56.291437: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291699: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:41:56.291712: I tensorflow/core/common_runtime/gpu/gpu

BI took: 3.9502 seconds


In [50]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    p = a;
    p = inv_logit(p);
    pulled_left ~ binomial( 1 , p );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 9.5s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.04 seconds.
  Adjust your expectations accordingly!


Pystan took: 9.7943 seconds


In [51]:
pd.DataFrame(
    {
        "tfp": m11_1.summary(round_to='none')['mean'],
        "pystan": [df.a.mean()]
    })

,tfp,pystan
a[0],0.321594,0.323123


##  5. Binomial with indices
### Output comparaisons

In [52]:
from src.main import*
d = pd.read_csv('./data/chimpanzees.csv', sep = ';')
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

formula = dict(
    main = 'pulled_left ~ Binomial(1 , p )' ,
    likelihood = 'p ~ a[actor] + b[treatment]' ,
    prior1 = 'a ~ Normal(0, 1.5)',
    prior2 = 'b ~ Normal(0, 0.5)'
)

start = tm.time()
m11_4 = model(formula, d, float = 32)
m11_4.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")


2024-03-12 08:42:21.132350: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132433: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:42:21.132634: I tensorflow/core/common_runtime/gpu/gpu

BI took: 6.8511 seconds


In [53]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
    array[504] int treatment;
    array[504] int actor;
}
parameters{
    vector[7] a;
    vector[4] b;
}
model{
    vector[504] p;
    b ~ normal( 0 , 0.5 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
generated quantities{
    vector[504] log_lik;
    vector[504] p;
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:504 ) log_lik[i] = binomial_lpmf( pulled_left[i] | 1 , p[i] );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int),
    'treatment' : d["treatment"].values.astype(int) +1,
    'actor' : d["actor"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.6s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 3.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.39 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.44 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.44 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 4.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.43 seconds.
  Adjust your expectations accordingly!


Pystan took: 15.2016 seconds


In [54]:
pd.DataFrame(
    {
        "tfp": m11_4.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), df['b.3'].mean(), df['b.4'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean(), df['a.5'].mean(), df['a.6'].mean(), df['a.7'].mean()]
    })

,tfp,pystan
b[0],-0.034848,-0.045185
b[1],0.475208,0.475823
b[2],-0.386461,-0.387507
b[3],0.366866,0.364700
a[0],-0.444979,-0.443071
a[1],3.917305,3.890319
a[2],-0.739946,-0.744365
a[3],-0.741094,-0.743383
a[4],-0.447257,-0.443050
a[5],0.482430,0.483948


## 6. Poisson
### Output comparaison

In [55]:
from src.main import*
d = pd.read_csv('./data/Kline.csv', sep = ';')
d["P"] = d.population.pipe(np.log).pipe(lambda x: (x - x.mean()) / x.std())
d["cid"] = (d.contact == "high").astype(int)
d['pLog'] = tf.math.log(d.P).numpy()
formula = dict(main = 'total_tools ~ Poisson(log_rate = lambda)',
               likelihood = 'lambda ~ alpha[cid] + beta[cid]*P',
               prior1 = 'alpha ~ Normal(3,0.5)',
               prior2 = 'beta ~ Normal(0,0.2)')
start = tm.time()
m11_10 = model(formula, d)
m11_10.fit(observed_data = dict(total_tools =d.total_tools.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:43:18.468534: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468588: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468601: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468813: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:18.468826: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.6212 seconds


In [56]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code = """ 
data{
    array[10] int T;
    vector[10] P;
    array[10] int cid;
}
parameters{
    vector[2] a;
    vector[2] b;
}
model{
    vector[10] lambda;
    b ~ normal( 0 , 0.2 );
    a ~ normal( 3 , 0.5 );
    for ( i in 1:10 ) {
       lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
       lambda[i] = exp(lambda[i]);
    }
    T ~ poisson( lambda );
}
generated quantities{
    vector[10] log_lik;
    vector[10] lambda;
    for ( i in 1:10 ) {
        lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
        lambda[i] = exp(lambda[i]);
    }
    for ( i in 1:10 ) log_lik[i] = poisson_lpmf( T[i] | lambda[i]);
    
}
"""

data = {
    'T' : d["total_tools"].values.astype(int),
    'P' : d["P"].values.astype(float),
    'cid' : d["cid"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.8s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!


Pystan took: 12.1948 seconds


In [57]:
pd.DataFrame(
    {
        "tfp": m11_10.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), 
                   df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
beta[0],0.377117,0.377000
beta[1],0.191245,0.193386
alpha[0],3.319345,3.319936
alpha[1],3.605054,3.609570


## 7. Negative binomial

In [58]:
from src.main import*
num_days = 30
y = tfd.Poisson(rate=1.5).sample((num_days,))
num_weeks = 4
y_new = tfd.Poisson(rate=0.5 * 7).sample((num_weeks,))
y_all = np.concatenate([y, y_new])
exposure = np.concatenate([np.repeat(1, 30), np.repeat(7, 4)])
monastery = np.concatenate([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame.from_dict(dict(y=y_all, days=exposure, monastery=monastery))
d["log_days"] = d.days.pipe(np.log)


formula = dict(main = 'y ~ Poisson(log_rate = lambda)',
               likelihood = 'lambda ~ log_days + alpha +  beta * monastery',
               prior1 = 'alpha ~ Normal(0,1)',
               prior2 = 'beta ~ Normal(0,1)')
start = tm.time()
m11_12 = model(formula, d, float=32)
m11_12.fit(observed_data = dict(y =d.y.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:43:55.540283: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:55.540355: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:55.540372: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:55.540498: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:43:55.540504: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.2123 seconds


In [31]:
d.head()

,y,days,monastery,log_days
0,3.0,1.0,0.0,0.0
1,1.0,1.0,0.0,0.0
2,2.0,1.0,0.0,0.0
3,1.0,1.0,0.0,0.0
4,3.0,1.0,0.0,0.0


In [39]:
import stan
import nest_asyncio
nest_asyncio.apply()

stan_code =""" 
data{
    array[34] int days;
    array[34] int y;
    array[34] int monastery;
    vector[34] log_days;
}
parameters{
    real a;
    real b;
}
model{
    vector[34] lambda;
    b ~ normal( 0 , 1 );
    a ~ normal( 0 , 1 );
    for ( i in 1:34 ) {
        lambda[i] = log_days[i] + a + b * monastery[i];
        lambda[i] = exp(lambda[i]);
    }
    
    y ~ poisson( lambda );
}
"""

data = {
    'days' : d["days"].values.astype(int),
    'y' : d["y"].values.astype(int),
    'monastery' : d["monastery"].values.astype(int) +1,
    'log_days' : d["log_days"].values.astype(float),
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.3s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.15 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.7091 seconds


## 8. Multinomial

## 9. Beta binomial

## 10. Zero inflated outcomes

## 11. Varying interceps

## 12. Varying effects

# Todo, param in model 2 to get in model 1

In [70]:
formula = dict(main = 'y~Normal(m,s)',
            likelihood = 'm ~  alpha + beta*z',
            prior1 = 's~Exponential(1)',
            prior2 = 'alpha ~ Normal(0,1)',
            prior3 = 'beta ~ Normal(0,1)',
            
            main1 = 'z~Normal(m2,s2)',
            likelihood2 = 'm2 ~ alpha2 + beta2',
            prior4 = 's2~Exponential(1)',
            prior5 = 'alpha2 ~ Normal(0,1)',
            prior6 = 'beta2 ~ Normal(0,1)') 

self = model()
self.f = formula
self.get_model_info()
self.merge()
self.write_priors()
self.write_mains()
self.build_tensor()
self.main_dict

2024-03-11 16:20:35.669067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 16:20:35.669252: I tensorflow/core/common_runtime/gpu/gpu

{'y': "lambda s, alpha, beta : tfd.Independent(tfd.Normal(alpha+beta*z,s, name ='main'), reinterpreted_batch_ndims=1)",
 'z': "lambda s2, alpha2, beta2 : tfd.Independent(tfd.Normal(alpha2+beta2,s2, name ='main1'), reinterpreted_batch_ndims=1)"}

In [86]:
self.mains[key]['likelihood(s)']

{'likelihood': {'input': 'm ~  alpha + beta*z',
  'output': 'm',
  'formula': 'alpha+beta*z',
  'args': array(['alpha', 'beta', 'z'], dtype='<U5'),
  'likelihood(s)': {},
  'params': {'args': ['alpha', 'beta', 'z'], 'kwargs': {}},
  'distribution(s)': [],
  'prior(s)': {'prior2': {'input': 'alpha ~ Normal(0,1)',
    'output': 'alpha',
    'formula': 'tfd.Normal(0,1)',
    'args': array(['0', '1', 'Normal'], dtype='<U6'),
    'likelihood(s)': {},
    'params': {'args': ['0', '1', 'Normal'], 'kwargs': {}},
    'distribution(s)': ['tfd.Normal'],
    'prior(s)': {}},
   'prior3': {'input': 'beta ~ Normal(0,1)',
    'output': 'beta',
    'formula': 'tfd.Normal(0,1)',
    'args': array(['0', '1', 'Normal'], dtype='<U6'),
    'likelihood(s)': {},
    'params': {'args': ['0', '1', 'Normal'], 'kwargs': {}},
    'distribution(s)': ['tfd.Normal'],
    'prior(s)': {}}}}}

In [84]:
key = 'main'
len(self.mains[key]) 
text = "lambda "
lambda_params = []
len(self.mains[key]['prior(s)']) 
for k in self.mains[key]['prior(s)'].keys():
    text = text + self.mains[key]['prior(s)'][k]["output"] + ', '
    lambda_params.append(self.mains[key]['prior(s)'][k]["output"])

len(self.mains[key]['likelihood(s)'])
for a in self.mains[key]['likelihood(s)'].keys():
                       # Check its priors and add them to lambda
                       if len(self.mains[key]['likelihood(s)'][a]['prior(s)']) > 0:
                           for b in  self.mains[key]['likelihood(s)'][a]['prior(s)'].keys():
                               if  self.mains[key]['likelihood(s)'][a]['prior(s)'][b]["output"] not in lambda_params:
                                   text = text + self.mains[key]['likelihood(s)'][a]['prior(s)'][b]["output"]  + ', '
text

'lambda s, alpha, beta, '